In [1]:
# !pip install pyspark
# !pip install  --upgrade ipython
import pyspark 
sc = pyspark.SparkContext('local[*]')

In [2]:
from datetime import datetime
from pyspark.sql.functions import col, udf,month,year
from pyspark.sql.types import DateType, IntegerType

In [3]:
from pyspark.sql import SparkSession
from operator import add

spark = SparkSession \
    .builder \
    .appName("Yelp Data Testing") \
    .getOrCreate()

df = spark.read.json("/Users/viralithakkar/Desktop/INF553/Project/yelp_dataset/yelp_academic_dataset_review.json")
df.printSchema()
#rdd = df.rdd.map(tuple)

#rdd = rdd.map(lambda r: r[3]).reduceByKey(add)

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [4]:
res = df.rdd.map(tuple)
res = res.map(lambda r: (r[0], 1)).reduceByKey(add)

In [5]:
res.count()

188593

In [6]:
res.take(10)

[('3Mx4renubPRnjHUw1n2UkA', 1030),
 ('yz66FIUPDKGhILDWzRLeKg', 255),
 ('qdC2h3MqCNussc5lhzL7gg', 29),
 ('ERCZtj8qxNxfXJrdXPdEsw', 3),
 ('VhQXJteVrG3L-wq8HlYFqw', 149),
 ('fL-b760btOaGa85OJ9ut3w', 2617),
 ('aIJqDWUt10iS45zYCr0KcQ', 266),
 ('qSsUXOWF9W4sekeHVekuug', 31),
 ('dNMYNKhAAKo5O4WfrAPzcQ', 8),
 ('dei67jnanWpRLRL7aRwqAA', 4)]

In [7]:
type(df)

pyspark.sql.dataframe.DataFrame

In [8]:
df = df.na.drop()
df = df.sort("date")

In [9]:
df.show(5)

+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|PD2MAlYYi9HCqPH7I...|   1|2004-10-12|    0|_CRpX4FGBkxie_1q0...|    5|Hole in the wall ...|     1|sE3ge33huDcNJGW3V...|
|u8C8pRvaHXg3PgDrs...|   0|2004-10-19|    1|0QHCY_55TFHHvyumE...|    4|Good stuff. Price...|     0|nkN_do3fJ9xekchVC...|
|oYMsq2Xvzw6UbrIlM...|   0|2004-10-19|    0|2F5J51OYtD49eyIUK...|    4|Love their pizza....|     0|nkN_do3fJ9xekchVC...|
|AtLv64FV-Pw6JuT3X...|   0|2004-10-19|    0|pho1XNCTeRxQVzWR_...|    4|Pokey Sticks are ...|     0|nkN_do3fJ9xekchVC...|
|GCRvrxMSC1nzShyM4...|   0|2004-10-19|    0|0O7kc-T1SrZpF84jM...|    5|New club in the H...|     0|yYSBB5q7bY-qSVvmM...|
+--------------------+----+-----

In [10]:
df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [11]:
df = df.withColumn("test", df['date'].cast(DateType()))
df = df.drop("date")
df = df.na.drop()


#df.show(5)

In [12]:

df = df.withColumn("month", month("test"))
df = df.withColumn("year", year("test"))


In [13]:
df.select("month","year").head(5)

[Row(month=10, year=2004),
 Row(month=10, year=2004),
 Row(month=10, year=2004),
 Row(month=10, year=2004),
 Row(month=10, year=2004)]

In [14]:
type(df)
df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: long (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- test: date (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)



In [15]:
df = df.filter(col('year') > 2010)

In [16]:
df.count()

5646630

In [17]:
df.take(1)

[Row(business_id='vAh6WNioOEw7G9tGhd_JQw', cool=4, funny=2, review_id='OXNVMnwKxJqUdKUiLNivtg', stars=4, text="So don't let the exterior fool you, this is not fast food, nor a buffet.\nWalking inside you are greeted by a helpful staff, and tables dressed with china and tablecloths. Totally surprising!\nWe ordered the beef chow fun dry which was excellent, the honey walnut shrimp, very tasty. Also ordered pot stickers which came with a delisious sauce.\nThe Jasmine Tea was delightful, and hot.\nReally an anomoly, talk about keeping your resturant on the down low...the signage is  a strip mall fast chinese food sign, we never would have walked in had it not been for Yelpers!\nWe will be back!", useful=9, user_id='eTmQbq9ZrL3mvhA55ppmHQ', test=datetime.date(2011, 1, 1), month=1, year=2011)]

In [18]:
temp = df.select(df.columns[:1])

In [19]:
temp.count()

5646630

In [20]:
temp.printSchema()

root
 |-- business_id: string (nullable = true)



In [21]:
temp = temp.select("business_id").distinct()

In [22]:
temp.count()

186568

In [23]:
res = df.groupBy("year","business_id").count()
res = res.withColumnRenamed("count", "total")
res = res.withColumn("total_int", res['total'].cast(IntegerType()))
res = res.filter(res["total_int"]>=20)
res = res.filter(res["year"] > 2010)

In [24]:
res = res.sort("year", ascending = True)
#res.sort("year", ascending = True).show( )

In [25]:
res.printSchema()

root
 |-- year: integer (nullable = true)
 |-- business_id: string (nullable = true)
 |-- total: long (nullable = false)
 |-- total_int: integer (nullable = false)



In [26]:
res.count()

59000

In [ ]:
df_review_spark = spark.read.json("/Users/viralithakkar/Desktop/INF553/Project/Yelp_Updated_Data/review.json")
df_review_spark.printSchema()

In [ ]:
df_business_spark = spark.read.json("/Users/viralithakkar/Desktop/INF553/Project/Yelp_Updated_Data/business.json")
df_business_spark.printSchema()

In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
# reading the JSON data using json.load()
file = "/Users/viralithakkar/Desktop/INF553/Project/Yelp_Updated_Data/business.json"
df_business = pd.read_json(file, lines = True)

In [3]:
df_business

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,3655 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",4JNXUYY8wbaaDmk3BPzlWw,"Steakhouses, Breakfast & Brunch, Restaurants, ...",Las Vegas,"{'Friday': '7:0-0:0', 'Monday': '7:0-23:0', 'S...",1,36.112827,-115.172581,Mon Ami Gabi,The Strip,89109,7968,4.0,NV
1,3570 S Las Vegas Blvd,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",RESDUcs7fIiihp38-d6_6g,"Sandwiches, Buffets, Breakfast & Brunch, Food,...",Las Vegas,"{'Friday': '7:30-22:0', 'Monday': '7:30-22:0',...",1,36.116113,-115.176222,Bacchanal Buffet,The Strip,89109,7866,4.0,NV
2,3708 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",K7lWdNUhCbcnEvI0NhGewg,"Buffets, Restaurants, Breakfast & Brunch",Las Vegas,"{'Friday': '8:0-22:0', 'Monday': '8:0-21:0', '...",1,36.109550,-115.176155,Wicked Spoon,The Strip,89109,6446,3.5,NV
3,3667 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",cYwJA2A6I12KNkm2rtXd5g,"Burgers, American (Traditional), Restaurants",Las Vegas,"{'Friday': '11:0-2:0', 'Monday': '11:0-0:0', '...",0,36.110724,-115.172169,Gordon Ramsay BurGR,The Strip,89109,5472,4.0,NV
4,3535 Las Vegas Blvd,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",f4x1YBxkLrZg652xt2KR5g,"Breakfast & Brunch, American (New), Restaurants",Las Vegas,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",1,36.118181,-115.171580,Hash House A Go Go,The Strip,89109,5382,4.0,NV
5,3667 Las Vegas Blvd S,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",DkYS3arLOhA8si5uUEmHOw,"Sandwiches, Wraps, Food, Caterers, Restaurants...",Las Vegas,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",1,36.109443,-115.171869,Earl of Sandwich,The Strip,89109,4981,4.5,NV
6,3131 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",2weQS-RnoOBhb1KsHKyoSQ,"Restaurants, Buffets",Las Vegas,"{'Friday': '7:30-21:30', 'Monday': '7:30-21:30...",1,36.126887,-115.165590,The Buffet,The Strip,89109,4240,3.5,NV
7,3708 Las Vegas Blvd S,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...",5LNZ67Yw9RD6nf4_UhXOjw,"Event Planning & Services, Casinos, Hotels, Ar...",Las Vegas,"{'Friday': '0:0-0:0', 'Monday': '0:0-0:0', 'Sa...",1,36.109755,-115.173877,The Cosmopolitan of Las Vegas,The Strip,89109,4097,4.0,NV
8,3600 S Las Vegas Blvd,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",ujHiaprwCQ5ewziu0Vi9rw,"Buffets, American (New), Restaurants",Las Vegas,"{'Friday': '7:0-22:0', 'Monday': '7:0-22:0', '...",1,36.113220,-115.176890,The Buffet at Bellagio,The Strip,89109,4091,3.5,NV
9,3708 Las Vegas Blvd S,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",iCQpiavjjPzJ5_3gPD5Ebg,"Pizza, Restaurants",Las Vegas,"{'Friday': '11:0-5:0', 'Monday': '11:0-5:0', '...",1,36.109837,-115.174212,Secret Pizza,The Strip,89109,4078,4.0,NV


In [4]:
df_business_rating = df_business[["business_id","stars"]]

In [5]:
df_business_rating

,business_id,stars
0,4JNXUYY8wbaaDmk3BPzlWw,4.0
1,RESDUcs7fIiihp38-d6_6g,4.0
2,K7lWdNUhCbcnEvI0NhGewg,3.5
3,cYwJA2A6I12KNkm2rtXd5g,4.0
4,f4x1YBxkLrZg652xt2KR5g,4.0
5,DkYS3arLOhA8si5uUEmHOw,4.5
6,2weQS-RnoOBhb1KsHKyoSQ,3.5
7,5LNZ67Yw9RD6nf4_UhXOjw,4.0
8,ujHiaprwCQ5ewziu0Vi9rw,3.5
9,iCQpiavjjPzJ5_3gPD5Ebg,4.0


In [6]:
total_businesses = len(df_business["business_id"].unique())
avg_allbBusiness_allUsers = df_business["stars"].mean()
avg_allbBusiness_allUsers

3.87

In [7]:
file = "/Users/viralithakkar/Desktop/INF553/Project/Yelp_Updated_Data/user.json"
df_users = pd.read_json(file, lines = True)

In [8]:
df_users

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,3.50,0,0,0,0,0,0,0,0,0,...,0,None,0,WlzI32M_6wmB72H78o6b4A,0,mindy,6,0,-2Pb5d2WBPtbyGT_be6NDQ,2009-04-29
1,2.47,0,0,0,0,0,0,1,0,0,...,1,None,0,"OYe02aZcjVDtziyMj1-0AA, nUZRKGwNxDrmQ_2vlVeVow...",1,Kim,15,61,-3bsS2i9xqjNnIA1fRnzIQ,2012-04-10
2,4.07,813,92,813,590,15,38,128,33,587,...,3052,"2010, 2016, 2008, 2015, 2007, 2011, 2014, 2013...",94,"CLKGEdkqwQT6oVJG_deC7g, sfow6eRRLPTQa4MEHmKbaQ...",2762,Linda,635,3215,-3i9bhfvrM3F1wsC9XIB8g,2005-08-07
3,4.55,0,0,0,0,0,0,0,0,0,...,0,None,0,None,0,Kady,36,3,-47g7LR58tpHlm7Bm24icg,2013-12-21
4,4.32,0,0,0,0,0,0,0,0,3,...,0,None,1,"tihKE7m6CUCaPg2mcc89Yw, wYPXdFqUh_D-y9j8tQVCdw...",0,Lisa,32,2,-8_yETBp70WiqqN-AxNm9Q,2008-09-29
5,4.10,5456,43,5456,3075,5,79,844,3330,2157,...,15512,"2012, 2017, 2018, 2013, 2014, 2016, 2015",174,"_owMBYfzT9dO7FXGXXzXqA, dYEHTYJeDJ4v8AUiExaVsw...",13579,Fran,1021,16172,-9da1xk7zgnnfO1uTVYGkA,2012-04-05
6,4.50,0,0,0,0,0,0,0,0,0,...,0,None,0,"2DAQ1MzKxw0nxuF3A0_luQ, 86tH0Y6hVrVzgD5HJGAvyg...",0,Couri,2,2,-CXUQegqiilOy--6TtajjA,2011-10-17
7,3.20,2,0,2,0,0,3,9,1,12,...,18,None,4,"2NWKoTSkkF-s80oxPjHZSw, k4tON7LfovRV2JwVUZfamQ...",50,James,184,53,-EZmRD-HXHz8G7D5RZBrWg,2011-12-20
8,3.90,8,0,8,4,0,3,5,1,18,...,7,"2012, 2013, 2014, 2015",18,"2qW-VO3UxZSU6HSHQ7fh7Q, RBZ_kMjowV0t6_nv2UKaDQ...",30,Jeff,313,40,-GKEFg_92pp0q842clS_Jw,2011-07-18
9,4.55,0,0,0,0,0,0,0,0,0,...,0,None,0,None,1,Lee,10,1,-QiFLgn_jBdTFcwkaV4WSg,2016-04-21


In [9]:
len(df_users["user_id"].unique())

461000

In [ ]:
file1 = "/Users/viralithakkar/Desktop/INF553/Project/Yelp_Updated_Data/review.json"

In [ ]:
reviews_df = pd.read_json(file1, lines = True)